In [54]:
import pandas as pd
import numpy as np
import vectorbt as vbt
from datetime import datetime
import os 
from numba import njit
from vectorbt.portfolio.enums import Direction, SizeType
import talib
import requests
import time


symbol = "XRP-USDT"
timeframe = "1day"
market_type = "spot"
start_time = "2020-01-01 00:00:00"
end_time = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
# end_time = "2021-06-10 00:00:00"

skip_fetch = False


In [55]:

if not skip_fetch:
    print("Fetching data...")
    def fetch_kucoin_candles_chunk(symbol, market_type="spot", timeframe="1min", start_time=None, end_time=None):
        time.sleep(0.2)  # Rate limiting
        base_url = "https://api.kucoin.com" if market_type.lower() == "spot" else "https://api-futures.kucoin.com"
        url = base_url + "/api/v1/market/candles"
        params = {"type": timeframe, "symbol": symbol.upper()}
        if start_time:
            params["startAt"] = int(time.mktime(time.strptime(start_time, "%Y-%m-%d %H:%M:%S")))
        if end_time:
            params["endAt"] = int(time.mktime(time.strptime(end_time, "%Y-%m-%d %H:%M:%S")))
        try:
            resp = requests.get(url, params=params, timeout=10)
            resp.raise_for_status()
            data = resp.json()
            if data.get("code") == "200000":
                return data["data"]
            else:
                raise Exception(f"KuCoin API error: {data}")
        except requests.exceptions.RequestException as e:
            raise Exception(f"Request failed: {str(e)}")

    def fetch_all_kucoin_candles(symbol, market_type="spot", timeframe="1min", start_time=None, end_time=None):
        chunks = []
        current_end = end_time
        start_timestamp = int(time.mktime(time.strptime(start_time, "%Y-%m-%d %H:%M:%S")))
        
        while True:
            try:
                chunk = fetch_kucoin_candles_chunk(symbol, market_type, timeframe, start_time, current_end)
                if not chunk:
                    print("No more data available")
                    break
                earliest_ts = int(chunk[-1][0])
                print(f"Fetched {len(chunk)} candles from {datetime.fromtimestamp(earliest_ts)}")
                chunks.extend(chunk)
                if earliest_ts <= start_timestamp:
                    print("Reached start time")
                    break
                current_end = time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime(earliest_ts - 60))
            except Exception as e:
                print(f"Error occurred: {str(e)}")
                break
        
        if not chunks:
            return []
        
        chunks.sort(key=lambda x: x[0])  # Sort by timestamp
        result = [candle for candle in chunks if start_timestamp <= int(candle[0]) <= int(time.mktime(time.strptime(end_time, "%Y-%m-%d %H:%M:%S")))]
        print(f"Total candles fetched: {len(result)}")
        return result


    data = fetch_all_kucoin_candles(symbol, market_type, timeframe, start_time, end_time)
    print(f"Final data length: {len(data)}")
    data
else:
    print("Skipping fetch...")

Fetching data...
Fetched 1500 candles from 2020-11-19 00:00:00
Fetched 323 candles from 2020-01-01 00:00:00
Reached start time
Total candles fetched: 1823
Final data length: 1823


In [56]:

# Assuming 'data' is the list of candlestick data
df = pd.DataFrame(data)
df[['timestamp', 'open', 'close', 'high', 'low']] = df[[0, 1, 2, 3, 4]]
df.drop([0, 1, 2, 3, 4, 5, 6], axis=1, inplace=True)

# Explicitly cast 'timestamp' to numeric type
df['timestamp'] = pd.to_numeric(df['timestamp'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', utc=True)

df.set_index('timestamp', inplace=True)
df.sort_index(inplace=True)
df[['open', 'close', 'high', 'low']] = df[['open', 'close', 'high', 'low']].astype(float)
df.info()
df

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1823 entries, 2020-01-01 00:00:00+00:00 to 2024-12-27 00:00:00+00:00
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    1823 non-null   float64
 1   close   1823 non-null   float64
 2   high    1823 non-null   float64
 3   low     1823 non-null   float64
dtypes: float64(4)
memory usage: 71.2 KB


,open,close,high,low
timestamp,,,,
2020-01-01 00:00:00+00:00,0.19293,0.19299,0.19524,0.19227
2020-01-02 00:00:00+00:00,0.19299,0.18740,0.19299,0.18579
2020-01-03 00:00:00+00:00,0.18743,0.19338,0.19387,0.18461
2020-01-04 00:00:00+00:00,0.19340,0.19315,0.19353,0.19118
2020-01-05 00:00:00+00:00,0.19304,0.19458,0.19712,0.19000
...,...,...,...,...
2024-12-23 00:00:00+00:00,2.20344,2.26173,2.27953,2.13119
2024-12-24 00:00:00+00:00,2.26174,2.32374,2.35096,2.21364
2024-12-25 00:00:00+00:00,2.32362,2.29736,2.33271,2.26164


## creating indicator to set signals long ONLY for simplicity

In [57]:
import pandas as pd
import numpy as np
import vectorbt as vbt

skip_cell = False

if not skip_cell:
    print("This cell is executed.")

    def momentum_strategy(close, high, low, atr_length, ema_length):
        # Calculate EMA
        EMA = vbt.IndicatorFactory.from_talib('EMA')
        ema = EMA.run(close, timeperiod=ema_length).real.to_numpy()

        # Calculate ATR
        ATR = vbt.IndicatorFactory.from_talib('ATR')
        atr = ATR.run(high, low, close, timeperiod=atr_length).real.to_numpy()

        # Define conditions
        is_bullish = close > ema

        def rolling_max(arr, window=7):
            # Create overlapping windows using stride tricks
            shape = (arr.size - window + 1, window)
            strides = (arr.strides[0], arr.strides[0])
            rolled = np.lib.stride_tricks.as_strided(arr, shape=shape, strides=strides)
            # Return the max over the last axis
            return np.max(rolled, axis=1)

        def compute_is_bearish_vol(high, low, atr, window=7):
            # Calculate rolling max for 'high'
            rm = rolling_max(high, window=window)
            # Pad with nan to align lengths
            rm_padded = np.concatenate([np.full(window-1, np.nan), rm])
            # Compare as in the original expression
            return (rm_padded - low) > (atr * 1.5)
        
        is_bearish_vol = compute_is_bearish_vol(high, low, atr)
        is_caution = is_bullish & is_bearish_vol | (close < ema)
        is_caution = ~is_bullish & (close < ema)
        
        # Define signals
        signal_buy = is_bullish & ~is_caution
        signal_sell = close < ema

        # Define conditions and corresponding values for signals
        conditions = [
            signal_buy,
            signal_sell
        ]

        values = [
            1,  # Buy signal
            -1  # Sell signal
        ]

        # Use np.select to apply the conditions and assign the corresponding values
        signal = np.select(conditions, values, default=0)  # 0: No signal (default value when none of the conditions are met)

        return signal

    # Create the indicator factory
    momentum_indicator = vbt.IndicatorFactory(
        class_name='MomentumStrategy',
        short_name='momentum',
        input_names=['Close', 'High', 'Low'],
        param_names=['atr_length', 'ema_length'],
        output_names=['signal']
    ).from_apply_func(momentum_strategy)

    # Define parameter ranges
    atr_length = np.arange(2, 10, 1)  # Range from 2 to 9 with step 1
    ema_length = np.arange(100, 301, 20)  # Range from 100 to 300 with step 20

    # Run the indicator
    signal = momentum_indicator.run(
        df['close'], df['high'], df['low'],
        atr_length=atr_length,
        ema_length=ema_length,
        param_product=True  
    )

    df_indicator = signal.signal

else:
    print("This cell is skipped.")

This cell is executed.


In [58]:
df_indicator

momentum_atr_length          2                                               \
momentum_ema_length        100  120  140  160  180  200  220  240  260  280   
timestamp                                                                     
2020-01-01 00:00:00+00:00    0    0    0    0    0    0    0    0    0    0   
2020-01-02 00:00:00+00:00    0    0    0    0    0    0    0    0    0    0   
2020-01-03 00:00:00+00:00    0    0    0    0    0    0    0    0    0    0   
2020-01-04 00:00:00+00:00    0    0    0    0    0    0    0    0    0    0   
2020-01-05 00:00:00+00:00    0    0    0    0    0    0    0    0    0    0   
...                        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2024-12-23 00:00:00+00:00  100  100  100  100  100  100  100  100  100  100   
2024-12-24 00:00:00+00:00  100  100  100  100  100  100  100  100  100  100   
2024-12-25 00:00:00+00:00  100  100  100  100  100  100  100  100  100  100   
2024-12-26 00:00:00+00:00  100  100  100  100  100  100  100  100  100  100   
2024-12-27 00:00:00+00:00  100  100  100  100  100  100  100  100  100  100   

momentum_atr_length        ...    9                                          \
momentum_ema_length        ...  120  140  160  180  200  220  240  260  280   
timestamp                  ...                                                
2020-01-01 00:00:00+00:00  ...    0    0    0    0    0    0    0    0    0   
2020-01-02 00:00:00+00:00  ...    0    0    0    0    0    0    0    0    0   
2020-01-03 00:00:00+00:00  ...    0    0    0    0    0    0    0    0    0   
2020-01-04 00:00:00+00:00  ...    0    0    0    0    0    0    0    0    0   
2020-01-05 00:00:00+00:00  ...    0    0    0    0    0    0    0    0    0   
...                        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2024-12-23 00:00:00+00:00  ...  100  100  100  100  100  100  100  100  100   
2024-12-24 00:00:00+00:00  ...  100  100  100  100  100  100  100  100  100   
2024-12-25 00:00:00+00:00  ...  100  100  100  100  100  100  100  100  100   
2024-12-26 00:00:00+00:00  ...  100  100  100  100  100  100  100  100  100   
2024-12-27 00:00:00+00:00  ...  100  100  100  100  100  100  100  100  100   

momentum_atr_length             
momentum_ema_length        300  
timestamp                       
2020-01-01 00:00:00+00:00    0  
2020-01-02 00:00:00+00:00    0  
2020-01-03 00:00:00+00:00    0  
2020-01-04 00:00:00+00:00    0  
2020-01-05 00:00:00+00:00    0  
...                        ...  
2024-12-23 00:00:00+00:00  100  
2024-12-24 00:00:00+00:00  100  
2024-12-25 00:00:00+00:00  100  
2024-12-26 00:00:00+00:00  100  
2024-12-27 00:00:00+00:00  100  

[1823 rows x 88 columns]

In [59]:
    # Create entries and exits based on the signals
    entries = df_indicator == 1
    exits = df_indicator == -1

    # Create portfolio
    pf = vbt.Portfolio.from_signals(
        df['close'],
        entries,
        exits,
        init_cash=10000,
        fees=0.001
    )

    # Display some portfolio performance metrics
    print("Total Return:", pf.total_return())
    print("\nOrder Records:")
    print(pf.orders.records_readable)

else:
    print("This cell is skipped.")

needed columns:  88
needed rows:  1823
shape: (1823, 88)

original shape:

 timestamp
2020-01-01 00:00:00+00:00    0.19299
2020-01-02 00:00:00+00:00    0.18740
2020-01-03 00:00:00+00:00    0.19338
2020-01-04 00:00:00+00:00    0.19315
2020-01-05 00:00:00+00:00    0.19458
                              ...   
2024-12-23 00:00:00+00:00    2.26173
2024-12-24 00:00:00+00:00    2.32374
2024-12-25 00:00:00+00:00    2.29736
2024-12-26 00:00:00+00:00    2.15595
2024-12-27 00:00:00+00:00    2.15956
Name: close, Length: 1823, dtype: float64


,0,1,2,3,4,5,6,7,8,9,...,78,79,80,81,82,83,84,85,86,87
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00+00:00,0.19299,0.19299,0.19299,0.19299,0.19299,0.19299,0.19299,0.19299,0.19299,0.19299,...,0.19299,0.19299,0.19299,0.19299,0.19299,0.19299,0.19299,0.19299,0.19299,0.19299
2020-01-02 00:00:00+00:00,0.18740,0.18740,0.18740,0.18740,0.18740,0.18740,0.18740,0.18740,0.18740,0.18740,...,0.18740,0.18740,0.18740,0.18740,0.18740,0.18740,0.18740,0.18740,0.18740,0.18740
2020-01-03 00:00:00+00:00,0.19338,0.19338,0.19338,0.19338,0.19338,0.19338,0.19338,0.19338,0.19338,0.19338,...,0.19338,0.19338,0.19338,0.19338,0.19338,0.19338,0.19338,0.19338,0.19338,0.19338
2020-01-04 00:00:00+00:00,0.19315,0.19315,0.19315,0.19315,0.19315,0.19315,0.19315,0.19315,0.19315,0.19315,...,0.19315,0.19315,0.19315,0.19315,0.19315,0.19315,0.19315,0.19315,0.19315,0.19315
2020-01-05 00:00:00+00:00,0.19458,0.19458,0.19458,0.19458,0.19458,0.19458,0.19458,0.19458,0.19458,0.19458,...,0.19458,0.19458,0.19458,0.19458,0.19458,0.19458,0.19458,0.19458,0.19458,0.19458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-23 00:00:00+00:00,2.26173,2.26173,2.26173,2.26173,2.26173,2.26173,2.26173,2.26173,2.26173,2.26173,...,2.26173,2.26173,2.26173,2.26173,2.26173,2.26173,2.26173,2.26173,2.26173,2.26173
2024-12-24 00:00:00+00:00,2.32374,2.32374,2.32374,2.32374,2.32374,2.32374,2.32374,2.32374,2.32374,2.32374,...,2.32374,2.32374,2.32374,2.32374,2.32374,2.32374,2.32374,2.32374,2.32374,2.32374
2024-12-25 00:00:00+00:00,2.29736,2.29736,2.29736,2.29736,2.29736,2.29736,2.29736,2.29736,2.29736,2.29736,...,2.29736,2.29736,2.29736,2.29736,2.29736,2.29736,2.29736,2.29736,2.29736,2.29736
